### 打开摄像头，检测移动物体，并用红线框出

In [1]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np  

#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture(0)  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
cv2.resizeWindow("DI-Video", 640, 480);


firstframe=None

while (camera.isOpened()):  

    ret,frame = camera.read()
    #print ret, frame
    
    if not ret:  
        break
        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if firstframe is None:  
        firstframe=gray  
        continue  
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
    #cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect
    
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)  
    key = cv2.waitKey(1)
     
    if key == ord("q"):  
        break  

camera.release()  
cv2.destroyAllWindows()  


### 打开视频文件，检测移动物体

In [5]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np  

#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
# cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
# cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
# cv2.resizeWindow("DI-Video", 640, 480);


firstframe=None

count=-1
frame_list=[]

while (camera.isOpened()):  

    ret,frame = camera.read()
    #print ret, frame
    
    if not ret:  
        break

        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if count%5==0:
        frame_list.append(gray)
    count+=1
    
    if firstframe is None:  
#         firstframe=gray\
        if count>100:
            firstframe=(np.average(frame_list,0)).astype(np.uint8)
            print((firstframe).dtype)
            print(gray.dtype)
#             break
        continue  
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
#     cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
    
    
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect
    
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)  
    key = cv2.waitKey(1)
     
    if key == ord("q"):  
        break  

camera.release()  
cv2.destroyAllWindows()  


uint8
uint8


### 从两幅图片提取特征点，并进行特征点的匹配

In [2]:
import cv2  
import numpy as np  
  
def drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch):  
    h1, w1 = img1_gray.shape[:2]  
    h2, w2 = img2_gray.shape[:2]  
  
    vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)  
    vis[:h1, :w1] = img1_gray  
    vis[:h2, w1:w1 + w2] = img2_gray  
  
    p1 = [kpp.queryIdx for kpp in goodMatch]  
    p2 = [kpp.trainIdx for kpp in goodMatch]  
  
    post1 = np.int32([kp1[pp].pt for pp in p1])  
    post2 = np.int32([kp2[pp].pt for pp in p2]) + (w1, 0)  
  
    for (x1, y1), (x2, y2) in zip(post1, post2):  
        cv2.line(vis, (x1, y1), (x2, y2), (0,0,255),thickness=1)  
  
    cv2.namedWindow("match",cv2.WINDOW_NORMAL)  
    cv2.imshow("match", vis)  
  
img1_gray = cv2.imread("moving3_pic1.JPG")  
img2_gray = cv2.imread("moving3_pic2.JPG")  
  
sift = cv2.xfeatures2d.SIFT_create()
#sift = cv2.SURF()  
  
kp1, des1 = sift.detectAndCompute(img1_gray, None)  
kp2, des2 = sift.detectAndCompute(img2_gray, None)  
print(len(kp1))
print(len(kp2))
  
# BFmatcher with default parms  
bf = cv2.BFMatcher(cv2.NORM_L2)  
matches = bf.knnMatch(des1, des2, k = 2)  
  
goodMatch = []  
for m,n in matches:  
    if m.distance/n.distance < 0.5:  
        goodMatch.append(m)  
drawMatchesKnn_cv2(img1_gray,kp1,img2_gray,kp2,goodMatch[:20])  

print(len(goodMatch))
  
cv2.waitKey(0)  
cv2.destroyAllWindows()

36
28
3


### 从视频中获取特征点的移动轨迹，未完成

In [3]:
# -*- coding: utf-8 -*-
import cv2  
import numpy as np  

#打开USB摄像头
#camera=cv2.VideoCapture(0)  

#获得视频的格式  
camera= cv2.VideoCapture("moving3.mp4")  

#以下fps，size用不了
#获得码率及尺寸  
#fps = camera.get(cv2.CV_CAP_PROP_FPS)  
#size = (int(camera.get(cv2.CV_CAP_PROP_FRAME_WIDTH)), int(camera.get(cv2.CV_CAP_PROP_FRAME_HEIGHT)))  

#指定写视频的格式, I420-avi, MJPG-mp4  
#videoWriter = cv2.VideoWriter('oto_other.mp4', cv2.cv.CV_FOURCC('M', 'J', 'P', 'G'), fps, size)  

#设置窗口大小，缺省的过大
cv2.namedWindow("Video",0);
# cv2.resizeWindow("Video", 800, 600);
cv2.namedWindow("BW-Video",0);
# cv2.resizeWindow("BW-Video", 640, 480);
cv2.namedWindow("DI-Video",0);
# cv2.resizeWindow("DI-Video", 640, 480);


firstframe=None

count=-1
frame_list=[]
frame=None
frame_raw=None
last_frame=None

while (camera.isOpened()):  
    if frame_raw is not None:
        last_frame=frame_raw.copy()
    ret,frame = camera.read()
    if frame is not None:
        frame_raw=frame.copy()
    #print ret, frame
    
    if not ret:  
        break

        
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)  
    gray=cv2.GaussianBlur(gray,(21,21),0)  
    
    if count%5==0:
        frame_list.append(gray)
    count+=1
    
    if firstframe is None:  
#         firstframe=gray\
        if count>100:
            firstframe=(np.average(frame_list,0)).astype(np.uint8)
            print((firstframe).dtype)
            print(gray.dtype)
#             break
        continue  
    
    
    #获得与背景的差值，背景很重要，最好是除了移动的主体之外，什么都不懂，且背景中没有移动的主体
    frameDelta = cv2.absdiff(firstframe,gray)  
    
    #获得黑白的图片
    thresh = cv2.threshold(frameDelta, 65, 255, cv2.THRESH_BINARY)[1]  
    thresh = cv2.dilate(thresh, None, iterations=2)  
    
    #获得移动物体的轮廓，注意opencv3.0以后，返回参数的位置不同，轮廓在第2位
    cnts, hierarchy, rr = cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  
    #print cnts
    
    #可以直接画出轮廓，但效果不如方框
#     cv2.drawContours(frame, hierarchy, -1, (0, 255, 0), 3)
    
    
 
    
    for c in hierarchy:
    
        # if the contour is too small, ignore it
        # 轮廓面积
        #area = cv2.contourArea(c)
        #print area
        # 周长，或者说，弧长；第二个参数的True表示该轮廓是否封闭
        #perimeter = cv2.arcLength(c,True)
        #sprint  perimeter
        
        if cv2.contourArea(c) <= 8000:
            continue
    
        # compute the bounding box for the contour, draw it on the frame,
        # and update the text
        
        # 计算轮廓的边界框，在当前帧中画出该框
        x,y,w,h = cv2.boundingRect(c)        
            
        #x,y,w,h=cv2.boundingRect(thresh)  
        frame=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3)  
        
        # 计算的最小轮廓
        #rect = cv2.minAreaRect(c)
        #print rect

    if last_frame is not None and count%10==0 and len(hierarchy)!=0:
        cv2.imwrite("shot"+str(count)+".png",last_frame)
        sift = cv2.xfeatures2d.SIFT_create() 
        kp1, des1 = sift.detectAndCompute(frame_raw, None)  
        kp2, des2 = sift.detectAndCompute(last_frame, None)  
        if len(kp1)!=0 and len(kp2)!=0:
            bf = cv2.BFMatcher(cv2.NORM_L2)  
            matches = bf.knnMatch(des1, des2, k = 2)  
            print(len(matches))
    
    cv2.imshow(u"Video", frame)  
    cv2.imshow(u"BW-Video", thresh)  
    cv2.imshow(u"DI-Video", frameDelta)  
    key = cv2.waitKey(1)
     
    if key == ord("q"):  
        break  
        
camera.release()  
cv2.destroyAllWindows()  


uint8
uint8
47
66
52
56
18
45
45
78
39
22
59
42
40
48
61
44
52
43
3
